In [ ]:
using QuadGK
using Plots

2次元の積分を入れ子の1次元積分で実行してみよう。

$$
\int dx dy~f(x, y) = \int dy~g(y),
$$

where

$$
g(y) \equiv \int dx~f(x, y).
$$

各軸の積分を1次元積分を使って、積分していくことで高精度な積分が可能になります。
もし、グリーン関数の波数積分に興味を持っている人は、[この論文](https://arxiv.org/abs/2211.12959)を読んでみると面白いと思います。

In [ ]:
function quad2d(f, xrange, yrange)
    # ...はタプルを複数の引数に展開する
    g(y) = quadgk(x -> f(x, y), xrange...)[1]
    return quadgk(g, yrange...) # 積分値とエラーを返す
end

## 2次元のガウス積分

In [ ]:
f(x, y) = exp(-(x^2 + y^2))

# エラーの推定値は一番外の積分に対するもの
result, error = quad2d(f, (-Inf, Inf), (-Inf, Inf))

println("Result: ", result)
println("Estimated error: ", error)
println("Actual error: ", result - π)

## 2次元のグリーン関数の波数積分

2次元の最近接tight bindingモデルの松原グリーン関数を考えてみよう。

$$
G(i\omega_0, k_x, k_y) = \frac{1}{i\omega_0 - \epsilon(k_x, k_y) + \epsilon_\mathrm{F} },
$$

where

$$
\epsilon(k) = 2 \cos(k_x) + 2 \cos(k_y),
$$

$$
\omega_0 = \pi T
$$

ここで, $T$は温度, $\epsilon_\mathrm{F}$はフェルミエネルギー。

In [ ]:
T = 0.01
ϵF = 1.0
ϵ(kx, ky) = 2(cos(kx) + cos(ky))
gk(kx, ky) = 1/(im * π * T - ϵ(kx, ky) + ϵF)

In [ ]:
nk = 1000

ks = collect(LinRange(-π, π, nk+1))[1:end-1]

gkdata = gk.(reshape(ks, nk, :), reshape(ks, :, nk))
;

In [ ]:
heatmap(abs.(gkdata))

In [ ]:
result_gk, error_gk = quad2d(gk, (-π, π), (-π, π))

println("Result: ", result_gk)
println("Estimated error: ", error_gk)

In [ ]:
result_gk_grid = sum(gkdata) * (2π/nk)^2

## 発展課題

2次元の波数積分を他の手法で行ってみましょう。例えば、`Cubature.jl`はnestした1次元積分ではなく、領域を適応的に分割して積分します。
どちらの手法が性能が良いか測ってみましょう (実行時間、関数の評価回数)。